# IF Analysis: Dual Channel Segmentation (Production Version)

Automated batch processing for immunofluorescence dual-channel images:
- **Channel 1 (Cells):** Cellpose cyto3 segmentation
- **Channel 1 (Proteins):** Blob detection with shape filtering
- **Channel 2 (Proteins):** Cellpose nuclei segmentation with size filtering

**Optimized Parameters:**
- Ch1 Blob Detection: min_sigma=0.5, max_sigma=2.0, threshold=0.015, overlap=0.001
- Shape Filter: min_circularity=0.6 (removes elongated lines)
- Size Filter: 0.5x-2.0x average size

---

## 1. Setup and Imports

In [7]:
import numpy as np
import tifffile
from cellpose import models
import torch
from pathlib import Path
import cv2
from scipy import ndimage
from skimage import filters
from skimage.feature import blob_log
from skimage.measure import regionprops
import pandas as pd

# Check for GPU
use_gpu = torch.cuda.is_available()
print(f"GPU Available: {use_gpu}")

GPU Available: True


## 2. Configuration

Set your folder path containing 2-channel TIF files.

In [8]:
# Input folder containing TIF files
FOLDER_PATH = r"S:\micro\ts2625\eh2888\lem\20260128_EVE_LargeImage\tiff\quantification"

# Get all TIF files
tif_files = sorted(Path(FOLDER_PATH).glob("*.tif")) + sorted(Path(FOLDER_PATH).glob("*.tiff"))

print(f"Found {len(tif_files)} TIF files")
if tif_files:
    for i, f in enumerate(tif_files[:5], 1):  # Show first 5
        print(f"  {i}. {f.name}")
    if len(tif_files) > 5:
        print(f"  ... and {len(tif_files)-5} more")
else:
    print("ERROR: No TIF files found!")

Found 23 TIF files
  1. DI_Gastro_0002_00001-03584-05314.tif
  2. DI_Gastro_0002_00001-05940-03286.tif
  3. DI_Gastro_0004_00001-05012-07199.tif
  4. DI_Gastro_0004_00001-05015-05372.tif
  5. DI_Gastro_0004_00001-06675-05268.tif
  ... and 18 more


## 3. Initialize Cellpose Models

In [9]:
# Model for large cells (Channel 1)
model_cells = models.CellposeModel(gpu=use_gpu, model_type='cyto3')

# Model for small protein markers (Channel 2)
model_protein = models.CellposeModel(gpu=use_gpu, model_type='nuclei')

print("✓ Models loaded")

model_type argument is not used in v4.0.1+. Ignoring this argument...
model_type argument is not used in v4.0.1+. Ignoring this argument...


✓ Models loaded


## 4. Helper Functions

Define reusable functions for filtering and detection.

In [10]:
def filter_masks_by_size(masks, min_ratio=0.5, max_ratio=2.0, verbose=False):
    """Filter masks based on size relative to average."""
    unique_labels = np.unique(masks)
    unique_labels = unique_labels[unique_labels != 0]
    
    if len(unique_labels) == 0:
        return masks, {'original_count': 0, 'filtered_count': 0, 'removed_count': 0}
    
    # Calculate sizes
    sizes = np.array([np.sum(masks == label_id) for label_id in unique_labels])
    avg_size = np.mean(sizes)
    min_size = avg_size * min_ratio
    max_size = avg_size * max_ratio
    
    # Filter
    filtered_masks = np.zeros_like(masks)
    new_label = 1
    for label_id, size in zip(unique_labels, sizes):
        if min_size <= size <= max_size:
            filtered_masks[masks == label_id] = new_label
            new_label += 1
    
    stats = {
        'original_count': len(unique_labels),
        'filtered_count': new_label - 1,
        'removed_count': len(unique_labels) - (new_label - 1)
    }
    
    if verbose:
        print(f"    Size filter: {stats['original_count']} → {stats['filtered_count']} "
              f"(removed {stats['removed_count']})")
    
    return filtered_masks, stats


def detect_proteins_blob(ch_cells, masks_cells, sigma=1, min_sigma=0.5, max_sigma=2.0, 
                         blob_threshold=0.015, overlap=0.001, background_offset=400,
                         min_circularity=0.6):
    """
    Detect small proteins using blob detection with shape filtering.
    
    Returns:
    --------
    masks : Filtered segmentation masks (shape + size filtered)
    processed_img : Background-subtracted and blurred image
    """
    # Calculate background
    background_mask = (masks_cells == 0)
    background_intensity = np.mean(ch_cells[background_mask]) + background_offset
    
    # Subtract background
    img_bg_subtracted = ch_cells.astype(np.float32) - background_intensity
    img_bg_subtracted = np.clip(img_bg_subtracted, 0, None)
    
    # Apply Gaussian blur
    img_processed = filters.gaussian(img_bg_subtracted, sigma=sigma, preserve_range=True)
    img_processed = img_processed.astype(np.uint16)
    
    # Normalize for blob detection
    img_norm = img_processed.astype(np.float32)
    if img_norm.max() > 0:
        img_norm = img_norm / img_norm.max()
    
    # Detect blobs
    blobs = blob_log(img_norm, min_sigma=min_sigma, max_sigma=max_sigma, 
                     threshold=blob_threshold, overlap=overlap)
    
    # Convert blobs to masks
    masks_raw = np.zeros(ch_cells.shape, dtype=np.uint16)
    for idx, blob in enumerate(blobs, 1):
        y, x, sigma_detected = blob
        radius = int(sigma_detected * np.sqrt(2))
        
        # Create circular mask
        yy, xx = np.ogrid[:ch_cells.shape[0], :ch_cells.shape[1]]
        circle_mask = (xx - x)**2 + (yy - y)**2 <= radius**2
        masks_raw[circle_mask] = idx
    
    # Filter by shape (remove elongated lines)
    if masks_raw.max() > 0 and min_circularity > 0:
        regions = regionprops(masks_raw)
        masks = np.zeros_like(masks_raw)
        new_label = 1
        
        for region in regions:
            area = region.area
            perimeter = region.perimeter
            
            if perimeter > 0:
                circularity = 4 * np.pi * area / (perimeter ** 2)
            else:
                circularity = 0
            
            # Keep only round objects
            if circularity >= min_circularity:
                masks[masks_raw == region.label] = new_label
                new_label += 1
    else:
        masks = masks_raw
    
    return masks, img_processed


def create_overlay(image, masks, color=(0, 255, 0), thickness=2):
    """Create overlay image with mask contours."""
    # Convert to RGB
    overlay = np.stack([image, image, image], axis=-1)
    if overlay.max() > 0:
        overlay = (255 * (overlay / overlay.max())).astype(np.uint8)
    else:
        overlay = overlay.astype(np.uint8)
    
    # Draw contours
    unique_labels = np.unique(masks)
    unique_labels = unique_labels[unique_labels != 0]
    
    for label_id in unique_labels:
        mask_region = (masks == label_id).astype(np.uint8)
        contours, _ = cv2.findContours(mask_region, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(overlay, contours, -1, color, thickness)
    
    return overlay

## 5. Batch Processing

Process all TIF files with optimized parameters.

**Processing Steps:**
1. Segment large cells (Ch1, Cellpose cyto3, diameter=120)
2. Detect Ch1 proteins (Blob detection with shape filtering)
3. Filter Ch1 proteins by size
4. Segment Ch2 proteins (Cellpose nuclei, diameter=6)
5. Filter Ch2 proteins by size
6. Create overlays and save all outputs

In [11]:
if tif_files:
    print(f"\n{'='*70}")
    print(f"BATCH PROCESSING: {len(tif_files)} files")
    print(f"{'='*70}\n")
    
    results_summary = []
    
    for file_idx, image_file in enumerate(tif_files, 1):
        filename_base = image_file.stem
        output_dir = image_file.parent
        
        print(f"[{file_idx}/{len(tif_files)}] {filename_base}")
        
        try:
            # Load image
            img = tifffile.imread(str(image_file))
            
            # Ensure shape is (height, width, channels)
            if img.ndim == 3 and img.shape[0] < 5:
                img = np.transpose(img, (1, 2, 0))
            
            # Extract channels
            ch_cells = img[..., 0]
            ch_protein = img[..., 1]
            
            # ========== CHANNEL 1: LARGE CELLS ==========
            masks_cells, _, _ = model_cells.eval(
                ch_cells,
                diameter=120,
                flow_threshold=0.4,
                cellprob_threshold=0.0,
                channels=[0, 0]
            )
            num_cells = masks_cells.max()
            print(f"  ✓ Cells: {num_cells}")
            
            # ========== CHANNEL 1: SMALL PROTEINS (BLOB DETECTION) ==========
            masks_ch1_proteins_raw, ch_cells_filtered = detect_proteins_blob(
                ch_cells,
                masks_cells,
                sigma=1,
                min_sigma=0.5,
                max_sigma=2.0,
                blob_threshold=0.015,
                overlap=0.001,
                background_offset=400,
                min_circularity=0.6
            )
            
            # Filter Ch1 protein masks by size
            masks_ch1_proteins, ch1_stats = filter_masks_by_size(
                masks_ch1_proteins_raw,
                min_ratio=0.5,
                max_ratio=2.0
            )
            num_ch1_proteins = ch1_stats['filtered_count']
            print(f"  ✓ Ch1 Proteins: {num_ch1_proteins} (blob detection, shape+size filtered)")
            
            # ========== CHANNEL 2: PROTEIN MARKERS ==========
            masks_protein_raw, _, _ = model_protein.eval(
                ch_protein,
                diameter=6,
                flow_threshold=0.5,
                cellprob_threshold=-1,
                channels=[0, 0]
            )
            
            # Filter Ch2 protein masks by size
            masks_protein, ch2_stats = filter_masks_by_size(
                masks_protein_raw,
                min_ratio=0.5,
                max_ratio=2.0
            )
            num_proteins = ch2_stats['filtered_count']
            print(f"  ✓ Ch2 Proteins: {num_proteins} (Cellpose, size filtered)")
            
            # ========== CREATE OVERLAYS ==========
            # Erode cell masks for better visualization
            eroded_cells = masks_cells.copy()
            for label_id in np.unique(masks_cells):
                if label_id == 0:
                    continue
                mask_region = (masks_cells == label_id).astype(np.uint8)
                eroded_region = ndimage.binary_erosion(mask_region, iterations=3).astype(np.uint8)
                eroded_cells[masks_cells == label_id] = 0
                eroded_cells[eroded_region > 0] = label_id
            
            overlay_cells = create_overlay(ch_cells, eroded_cells, color=(0, 255, 0), thickness=2)
            overlay_ch1_proteins = create_overlay(ch_cells_filtered, masks_ch1_proteins, color=(0, 255, 255), thickness=1)
            overlay_protein = create_overlay(ch_protein, masks_protein, color=(255, 0, 255), thickness=1)
            
            # ========== SAVE OUTPUTS ==========
            # Cell outputs
            tifffile.imwrite(str(output_dir / f'{filename_base}_ch1_cells_mask.tif'), 
                           eroded_cells.astype(np.uint16))
            tifffile.imwrite(str(output_dir / f'{filename_base}_ch1_cells_overlay.tif'), 
                           overlay_cells)
            
            # Ch1 protein outputs
            tifffile.imwrite(str(output_dir / f'{filename_base}_ch1_proteins_mask.tif'), 
                           masks_ch1_proteins.astype(np.uint16))
            tifffile.imwrite(str(output_dir / f'{filename_base}_ch1_proteins_overlay.tif'), 
                           overlay_ch1_proteins)
            tifffile.imwrite(str(output_dir / f'{filename_base}_ch1_bg_subtracted.tif'), 
                           ch_cells_filtered)
            
            # Ch2 protein outputs
            tifffile.imwrite(str(output_dir / f'{filename_base}_ch2_protein_mask.tif'), 
                           masks_protein.astype(np.uint16))
            tifffile.imwrite(str(output_dir / f'{filename_base}_ch2_protein_overlay.tif'), 
                           overlay_protein)
            
            print(f"  ✓ Saved all outputs\n")
            
            results_summary.append({
                'file': filename_base,
                'status': 'SUCCESS',
                'cells': num_cells,
                'ch1_proteins': num_ch1_proteins,
                'ch1_proteins_removed': ch1_stats['removed_count'],
                'ch2_proteins': num_proteins,
                'ch2_proteins_removed': ch2_stats['removed_count']
            })
            
        except Exception as e:
            print(f"  ✗ ERROR: {str(e)}\n")
            results_summary.append({
                'file': filename_base,
                'status': f'FAILED: {str(e)}',
                'cells': 0,
                'ch1_proteins': 0,
                'ch1_proteins_removed': 0,
                'ch2_proteins': 0,
                'ch2_proteins_removed': 0
            })
    
    # Print summary
    print(f"{'='*70}")
    print("PROCESSING COMPLETE")
    print(f"{'='*70}")
    
    successful = sum(1 for r in results_summary if r['status'] == 'SUCCESS')
    print(f"Total: {successful}/{len(tif_files)} files processed successfully\n")
    
    if successful > 0:
        print("Summary:")
        for result in results_summary:
            if result['status'] == 'SUCCESS':
                print(f"  {result['file']}: "
                      f"Cells={result['cells']}, "
                      f"Ch1_Proteins={result['ch1_proteins']}, "
                      f"Ch2_Proteins={result['ch2_proteins']}")
    
    if successful < len(tif_files):
        print(f"\n⚠ {len(tif_files) - successful} files failed")
        for result in results_summary:
            if result['status'] != 'SUCCESS':
                print(f"  {result['file']}: {result['status']}")
else:
    print("No TIF files to process. Check FOLDER_PATH.")

channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used



BATCH PROCESSING: 23 files

[1/23] DI_Gastro_0002_00001-03584-05314


Resizing is deprecated in v4.0.1+


  ✓ Cells: 366


channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used


...

  ✓ Cells: 36


channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used


  ✓ Ch1 Proteins: 994 (blob detection, shape+size filtered)


Resizing is deprecated in v4.0.1+


  ✓ Ch2 Proteins: 75 (Cellpose, size filtered)
  ✓ Saved all outputs

PROCESSING COMPLETE
Total: 22/23 files processed successfully

Summary:
  DI_Gastro_0002_00001-03584-05314: Cells=366, Ch1_Proteins=5020, Ch2_Proteins=541
  DI_Gastro_0002_00001-05940-03286: Cells=334, Ch1_Proteins=2755, Ch2_Proteins=769
  DI_Gastro_0004_00001-05012-07199: Cells=150, Ch1_Proteins=1062, Ch2_Proteins=186
  DI_Gastro_0004_00001-05015-05372: Cells=188, Ch1_Proteins=3844, Ch2_Proteins=295
  DI_Gastro_0004_00001-06675-05268: Cells=297, Ch1_Proteins=1, Ch2_Proteins=510
  Saline_Gastro_00001-04476-07542: Cells=158, Ch1_Proteins=103, Ch2_Proteins=478
  Saline_Gastro_00001-09822-04812: Cells=331, Ch1_Proteins=1502, Ch2_Proteins=755
  Saline_Gastro_00002-11874-04140: Cells=216, Ch1_Proteins=2038, Ch2_Proteins=469
  Saline_Gastro_0001_00001-05528-05766: Cells=223, Ch1_Proteins=260, Ch2_Proteins=753
  Saline_Gastro_0001_00001-08477-04522: Cells=344, Ch1_Proteins=1138, Ch2_Proteins=892
  Saline_Gastro_0002_00001-0

## 6. Export Summary to CSV

In [12]:
if 'results_summary' in locals() and results_summary:
    df = pd.DataFrame(results_summary)
    csv_path = Path(FOLDER_PATH) / 'IF_segmentation_summary.csv'
    df.to_csv(csv_path, index=False)
    
    print(f"✓ Summary saved to: {csv_path}")
    print(f"\nDataFrame ({len(df)} rows):")
    print(df.to_string())
else:
    print("No results to save. Run batch processing first.")

✓ Summary saved to: S:\micro\ts2625\eh2888\lem\20260128_EVE_LargeImage\tiff\quantification\IF_segmentation_summary.csv

DataFrame (23 rows):
                                    file                                                 status  cells  ch1_proteins  ch1_proteins_removed  ch2_proteins  ch2_proteins_removed
0       DI_Gastro_0002_00001-03584-05314                                                SUCCESS    366          5020                    10           541                   150
1       DI_Gastro_0002_00001-05940-03286                                                SUCCESS    334          2755                  1038           769                   171
2       DI_Gastro_0004_00001-05012-07199                                                SUCCESS    150          1062                   269           186                    52
3       DI_Gastro_0004_00001-05015-05372                                                SUCCESS    188          3844                    11           295       

## Output Files

For each input image `sample.tif`, the following files are generated:

**Cell Segmentation (Ch1):**
- `sample_ch1_cells_mask.tif` - Label mask for cells
- `sample_ch1_cells_overlay.tif` - RGB overlay with cell boundaries (green)

**Ch1 Protein Detection (Blob):**
- `sample_ch1_proteins_mask.tif` - Label mask for proteins
- `sample_ch1_proteins_overlay.tif` - RGB overlay with protein boundaries (cyan)
- `sample_ch1_bg_subtracted.tif` - Background-subtracted image

**Ch2 Protein Detection (Cellpose):**
- `sample_ch2_protein_mask.tif` - Label mask for proteins
- `sample_ch2_protein_overlay.tif` - RGB overlay with protein boundaries (magenta)

**Summary:**
- `IF_segmentation_summary.csv` - Quantification results for all images

---

## Notes

**Optimized Parameters Used:**
- Cell diameter: 120 pixels
- Ch1 Blob Detection: min_sigma=0.5, max_sigma=2.0, threshold=0.015, overlap=0.001
- Shape filtering: min_circularity=0.6 (removes elongated lines)
- Size filtering: 0.5x-2.0x average size for both channels
- Background offset: +400 intensity units

**To modify parameters, edit the values in Section 5 (Batch Processing).**

---

## Alternative Approach: ImageJ Macro for Protein Segmentation

Using Find Maxima method instead of Cellpose/blob detection:
- Rolling background subtraction (100 pixels)
- Gaussian blur (1 pixel)
- Find Maxima (threshold=1000)
- Process both Channel 1 and Channel 2


In [ ]:
# Save this ImageJ macro to a .ijm file and run it in ImageJ/Fiji

imagej_macro = """
// ImageJ Macro for Dual Channel Protein Segmentation
// Using Rolling Background Subtraction, Gaussian Blur, and Find Maxima
//
// Author: Generated for IF Analysis
// Date: 2026-02-06

// ========== CONFIGURATION ==========
// Set your input folder path here
inputFolder = "S:/micro/ts2625/eh2888/lem/20260128_EVE_LargeImage/tiff/quantification/";

// Parameters
rollingRadius = 100;    // Rolling ball radius for background subtraction
gaussianSigma = 1.0;    // Gaussian blur sigma
maxThreshold = 1000;    // Find Maxima threshold
outputType = "Segmented Particles";  // Or "Point Selection", "Count", "Maxima Within Tolerance", "Single Points"

// ========== MAIN PROCESSING ==========
setBatchMode(true);

// Get list of TIF files
list = getFileList(inputFolder);
tifFiles = newArray();
for (i = 0; i < list.length; i++) {
    if (endsWith(list[i], ".tif") || endsWith(list[i], ".tiff")) {
        tifFiles = Array.concat(tifFiles, list[i]);
    }
}

print("\\nFound " + tifFiles.length + " TIF files");
print("Processing started...\\n");

// Process each file
for (fileIdx = 0; fileIdx < tifFiles.length; fileIdx++) {
    filename = tifFiles[fileIdx];
    filepath = inputFolder + filename;
    basename = replace(filename, ".tif", "");
    basename = replace(basename, ".tiff", "");
    
    print("[" + (fileIdx+1) + "/" + tifFiles.length + "] Processing: " + filename);
    
    // Open image
    open(filepath);
    originalID = getImageID();
    originalTitle = getTitle();
    
    // Get dimensions
    getDimensions(width, height, channels, slices, frames);
    
    if (channels < 2) {
        print("  ERROR: Image must have at least 2 channels");
        close();
        continue;
    }
    
    // Split channels
    run("Split Channels");
    
    // ========== PROCESS CHANNEL 1 (Proteins) ==========
    selectWindow("C1-" + originalTitle);
    ch1_originalID = getImageID();
    
    // Duplicate for processing
    run("Duplicate...", "title=Ch1_Processing");
    
    // 1. Rolling background subtraction
    run("Subtract Background...", "rolling=" + rollingRadius);
    
    // 2. Gaussian blur
    run("Gaussian Blur...", "sigma=" + gaussianSigma);
    
    // Save background subtracted image
    saveAs("Tiff", inputFolder + basename + "_ch1_bg_subtracted.tif");
    ch1_processed = getTitle();
    
    // 3. Find Maxima
    run("Find Maxima...", "prominence=" + maxThreshold + " output=[" + outputType + "]");
    ch1_mask = getTitle();
    
    // Count objects
    run("Set Measurements...", "area mean min redirect=None decimal=3");
    run("Analyze Particles...", "size=0-Infinity display clear");
    ch1_count = nResults;
    
    // Save mask
    selectWindow(ch1_mask);
    saveAs("Tiff", inputFolder + basename + "_ch1_proteins_mask.tif");
    
    // Create overlay
    selectWindow(ch1_processed);
    run("Duplicate...", "title=Ch1_Overlay");
    run("RGB Color");
    
    selectWindow(basename + "_ch1_proteins_mask.tif");
    run("Duplicate...", "title=TempMask");
    setThreshold(1, 65535);
    run("Create Selection");
    
    if (selectionType() != -1) {
        selectWindow("Ch1_Overlay");
        run("Restore Selection");
        setForegroundColor(0, 255, 255); // Cyan
        run("Draw", "slice");
        run("Select None");
    }
    
    saveAs("Tiff", inputFolder + basename + "_ch1_proteins_overlay.tif");
    close();
    close("TempMask");
    
    print("  Ch1 Proteins: " + ch1_count + " objects detected");
    
    // Close Ch1 windows
    close(ch1_mask);
    close(ch1_processed);
    selectImage(ch1_originalID);
    close();
    
    // ========== PROCESS CHANNEL 2 (Proteins) ==========
    selectWindow("C2-" + originalTitle);
    ch2_originalID = getImageID();
    
    // Duplicate for processing
    run("Duplicate...", "title=Ch2_Processing");
    
    // 1. Rolling background subtraction
    run("Subtract Background...", "rolling=" + rollingRadius);
    
    // 2. Gaussian blur
    run("Gaussian Blur...", "sigma=" + gaussianSigma);
    
    // Save background subtracted image
    saveAs("Tiff", inputFolder + basename + "_ch2_bg_subtracted.tif");
    ch2_processed = getTitle();
    
    // 3. Find Maxima
    run("Find Maxima...", "prominence=" + maxThreshold + " output=[" + outputType + "]");
    ch2_mask = getTitle();
    
    // Count objects
    run("Analyze Particles...", "size=0-Infinity display clear");
    ch2_count = nResults;
    
    // Save mask
    selectWindow(ch2_mask);
    saveAs("Tiff", inputFolder + basename + "_ch2_protein_mask.tif");
    
    // Create overlay
    selectWindow(ch2_processed);
    run("Duplicate...", "title=Ch2_Overlay");
    run("RGB Color");
    
    selectWindow(basename + "_ch2_protein_mask.tif");
    run("Duplicate...", "title=TempMask2");
    setThreshold(1, 65535);
    run("Create Selection");
    
    if (selectionType() != -1) {
        selectWindow("Ch2_Overlay");
        run("Restore Selection");
        setForegroundColor(255, 0, 255); // Magenta
        run("Draw", "slice");
        run("Select None");
    }
    
    saveAs("Tiff", inputFolder + basename + "_ch2_protein_overlay.tif");
    close();
    close("TempMask2");
    
    print("  Ch2 Proteins: " + ch2_count + " objects detected");
    
    // Close Ch2 windows
    close(ch2_mask);
    close(ch2_processed);
    selectImage(ch2_originalID);
    close();
    
    print("  Saved all outputs\\n");
}

setBatchMode(false);
print("\\n========== PROCESSING COMPLETE ==========");
print("Total files processed: " + tifFiles.length);
print("\\nOutput files for each image:");
print("  - {basename}_ch1_bg_subtracted.tif");
print("  - {basename}_ch1_proteins_mask.tif");
print("  - {basename}_ch1_proteins_overlay.tif");
print("  - {basename}_ch2_bg_subtracted.tif");
print("  - {basename}_ch2_protein_mask.tif");
print("  - {basename}_ch2_protein_overlay.tif");
"""

# Save the macro to a file
macro_path = Path(FOLDER_PATH) / "IF_FindMaxima_DualChannel.ijm"
with open(macro_path, 'w') as f:
    f.write(imagej_macro)

print(f"✓ ImageJ macro saved to: {macro_path}")
print("\nTo use this macro:")
print("1. Open ImageJ/Fiji")
print("2. Go to Plugins > Macros > Edit...")
print("3. Open the saved .ijm file")
print("4. Update the 'inputFolder' path if needed")
print("5. Click 'Run' to process all TIF files in the folder")
print("\nThe macro will process both channels and save:")
print("  - Background-subtracted images (*_bg_subtracted.tif)")
print("  - Segmentation masks (*_proteins_mask.tif)")
print("  - RGB overlays (*_proteins_overlay.tif)")


### ImageJ Macro Parameters

You can customize these parameters at the top of the macro:

```java
rollingRadius = 100;    // Rolling ball radius (pixels)
gaussianSigma = 1.0;    // Gaussian blur sigma (pixels)
maxThreshold = 1000;    // Find Maxima prominence threshold
```

**Output Type Options for Find Maxima:**
- `"Segmented Particles"` - Creates label masks (recommended)
- `"Single Points"` - Creates single-pixel maxima
- `"Point Selection"` - Just marks the maxima locations
- `"Count"` - Only counts maxima
- `"Maxima Within Tolerance"` - Creates binary image

**Processing Steps:**
1. **Rolling Background Subtraction (100 pixels)**: Removes large-scale intensity variations
2. **Gaussian Blur (1 pixel)**: Smooths noise while preserving edges
3. **Find Maxima (1000 threshold)**: Detects local intensity peaks above threshold
4. **Segmentation**: Creates labeled masks for each detected object

**Output Files** (per input image):
- Channel 1:
  - `{name}_ch1_bg_subtracted.tif`
  - `{name}_ch1_proteins_mask.tif` (label mask)
  - `{name}_ch1_proteins_overlay.tif` (cyan boundaries)
- Channel 2:
  - `{name}_ch2_bg_subtracted.tif`
  - `{name}_ch2_protein_mask.tif` (label mask)
  - `{name}_ch2_protein_overlay.tif` (magenta boundaries)
